In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
#import necessary libraries
import pandas as pd
import h5py
import numpy as np
from scipy.stats import zscore,kstest
import random
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

import sys  # for importing one level above
sys.path.append("..")
from placecode.spatial_coding_functions import firing_rate_map, read_spatial, make_binary, filter_event_count, vector_sum, spiking_rate_map
from placecode.utils import open_file, open_dir
from placecode.analysis_info import ExpInfo, AnalysisParams

In [ ]:
save_results = False  # flag whether to save results into hdf5 file 

## Locate files

In [ ]:
fpath_expinfo = open_file("Select experiment info json file!")
exp_info = ExpInfo(fpath_expinfo)  # TODO: mount data folder! so we can access it

In [ ]:
# open analysis parameters, append mouse/session-specific data, save it later with results.
fpath_analysis_params = open_file("Select analysis parameters json file!")
analysis_params = AnalysisParams(fpath_analysis_params)
analysis_params.read_exp_info(exp_info)  # extract necessary experiment information for analysis

In [ ]:
# make sure the necessary files exist
assert os.path.exists(exp_info.fpath_caim)
assert os.path.exists(exp_info.fpath_loco)

In [ ]:
# select folder to save results
output_folder = open_dir("Select folder for output")  
# TODO: add output folder to analysis parameters? Create/check folder mouse_id -> condition, save results there

## Load data

In [ ]:
# load CaImAn data
with h5py.File(exp_info.fpath_caim, "r") as hf_caim:
    # temporal
    temporal_raw=hf_caim['estimates']['C'][()]
    n_components, n_frames = temporal_raw.shape
    resolution = hf_caim["dims"][()]
    # access a single temporal component as temporal_raw[i]

    # spatial
    resolution = hf_caim["dims"][()]
    A_data = hf_caim["estimates"]["A"]["data"][()]
    A_indices = hf_caim["estimates"]["A"]["indices"][()]
    A_indptr = hf_caim["estimates"]["A"]["indptr"][()]
    A_shape = hf_caim["estimates"]["A"]["shape"][()]
    spatial = read_spatial(A_data, A_indices, A_indptr, A_shape, n_components, resolution, unflatten=True)

In [ ]:
# load loco data
# TODO: include stripes, distance per round, etc. in loco data cut to scanner time frame. Missing in Martin's code?
#   check https://github.com/mitlabence/matlab-2p/issues/11
dict_loco = dict()
with h5py.File(exp_info.fpath_loco, "r") as hf_loco:
    for dset_name in hf_loco["inferred"]["belt_scn_dict"].keys():
        dtype = np.int16 if dset_name in ["round", "rounds", "stripes"] else np.float64
        dict_loco[dset_name] = hf_loco["inferred"]["belt_scn_dict"][dset_name][()].astype(dtype)
print(dict_loco.keys())


## Preprocess data

Create z-score of temporal components

In [ ]:
temporal_z = zscore(temporal_raw, axis=1) 

In [ ]:
from scipy.signal import find_peaks

Create binary trace

In [ ]:
temporal_binary = np.zeros(shape=temporal_raw.shape, dtype=bool)
for i_unit in range(n_components):
    idx_peaks = find_peaks(temporal_raw[i_unit], height=analysis_params.peak_threshold, distance=analysis_params.peak_distance)[0]
    temporal_binary[i_unit][idx_peaks] = 1 

### Filter rounds
Only use rounds where the total length adds up to the expected belt length

In [ ]:
expected_distance=exp_info.belt_length_mm
lv_rounds = dict_loco["rounds"].copy()  
lv_distPR = dict_loco["distance"].copy() # belt_scn_dict hast distance per round as distance, see issue above
lv_speed = dict_loco["speed"].copy()
n_rounds=lv_rounds.max()  # number of finished rounds
rounds = []
round_flags = np.zeros(n_rounds, dtype=np.int8)  # 1 if corresponding round included in analysis, 0 otherwise

for round in range(1,n_rounds+1):
    dist_current_round=lv_distPR[lv_rounds==round][-1]
    #print(dist_current_round)
    if abs(dist_current_round-expected_distance)<15:
        rounds.append(round-1)
        round_flags[round-1] = 1
    else:
        print(f"Not using {round}")


num_rounds=len(rounds)
print(f'Rounds (starting with 0): {rounds}\n Number of rounds used: {num_rounds}, total {n_rounds}')

# save as parameter a binary array on which round was included
analysis_params.rounds_included = round_flags




In [ ]:
np.sum(temporal_binary, axis=1)  # number of firing events per neuron (unfiltered)

### Filter forward-locomoting frames
Only use frames where mouse is running forward

In [ ]:
# filter the data
included_rounds_frames =  np.isin(dict_loco["rounds"], np.array(rounds) - 1)  # convert rounds to 0-indexing, filter to only those rounds that count
loco_frames = dict_loco["speed"] > 0
idx_filtered = np.logical_and(included_rounds_frames, loco_frames)

dict_loco_cut = dict()
for k in dict_loco:
    dict_loco_cut[k] = dict_loco[k][idx_filtered]
temporal_raw_cut = temporal_raw[:, idx_filtered]
temporal_z_cut = temporal_z[:, idx_filtered]
temporal_binary_cut = temporal_binary[:, idx_filtered]
rounds_cut = lv_rounds[idx_filtered]  # get corresponding round index
distPR_cut = lv_distPR[idx_filtered]
lv_speed_cut = lv_speed[idx_filtered]

included_frames = np.arange(0, n_frames)
included_frames_cut = included_frames[idx_filtered]

In [ ]:
np.sum(temporal_binary_cut, axis=1)  # the firing events per cell used directly for analysis

### Calculate spatial firing map

In [ ]:
n_bins = analysis_params.n_bins
n_units = n_components
analysis_params.n_units = n_units

In [ ]:
bin_borders = np.linspace(0, analysis_params.belt_length_mm, analysis_params.n_bins, endpoint=True)
#  get firing rate maps
frm_raw = firing_rate_map(temporal_raw_cut, dict_loco_cut["rounds"], dict_loco_cut["distance"], n_bins)
frm_z = firing_rate_map(temporal_z_cut, dict_loco_cut["rounds"], dict_loco_cut["distance"], n_bins)
# average over rounds
frm_raw_avg = np.mean(frm_raw, axis=1)
frm_z_avg = np.mean(frm_z, axis=1)
# calculate mean event map
# new firing rate map method: use binary temporal components
binary_spiking= spiking_rate_map(temporal_binary_cut, dict_loco_cut["rounds"], dict_loco_cut["distance"], n_bins) # make_binary(frm_raw,peak_threshold=analysis_params.peak_threshold,peak_distance=analysis_params.peak_distance)
# apply minimum event count threshold to each cell, i.e. detect "frequently firing" cells
# TODO: rename "frequently firing" cells to something better
n_events_threshold = analysis_params.n_events_threshold
i_cells_frequent_firing = filter_event_count(binary_spiking, n_events_threshold)
# get a percent value of the "frequently firing" cells
prc_frequent_firing_cells=100*len(i_cells_frequent_firing)/n_units
# TODO: save indices of cells that get accepted in this step 

### Check processed data
Plot locomotion (grey) and tmeporal component (green). Show frames included in analysis (blue horizontal lines) and firing events that are included (vertical lines color coded (10 separate colors) based on bins)

In [ ]:
bin_beginnings = np.linspace(0, analysis_params.belt_length_mm, analysis_params.n_bins, endpoint=False)  # the mm value of the beginning of each bin
cmap = mpl.colormaps["tab10"]
def plot_debug(i_cell):
    fig, axs = plt.subplots(2, 1, figsize=(12, 24))
    offset = 0.0
    axs[0].imshow(binary_spiking[i_cell])
    axs[0].set_ylim((-1, np.max(rounds)+1))
    for round in rounds:
        idx_current_round = lv_rounds == round
        temporal_current_round = temporal_raw[i_cell][idx_current_round]
        speed_current_round = lv_speed[idx_current_round]
        included_frames_current_round = idx_filtered[idx_current_round]
        dist_current_round = lv_distPR[idx_current_round]
        # get all spikes that occur during the included frames
        events_during_included_frames = np.logical_and(temporal_binary[i_cell][idx_current_round], included_frames_current_round)
        firing_current_round = np.nonzero(events_during_included_frames) 
        
        # get corresponding spatial bin for each firing event 
        # find first element greater than distance at which cell fired (index of next bin)
        # subtract 1 to find the bin index
        i_bins_events = np.searchsorted(bin_beginnings, dist_current_round[firing_current_round], side="right") - 1
        # get corresponding indices of color map 
        i_event_colors = i_bins_events%len(cmap.colors)
        # create locomotion segments as (loco_begin_frame, loco_end_frame)
        included_segments = []
        i_begin_segment = 0
        for i in range(1, len(included_frames_current_round)):
            # detect beginning of a segment: previous frame not included, current included
            if included_frames_current_round[i] and not included_frames_current_round[i-1]:
                i_begin_segment = i
            # detect end of a segment: previous frame included, current not included
            elif not included_frames_current_round[i] and included_frames_current_round[i-1]:
                i_end_segment = i-1
                # end of segment found: add segment to list
                included_segments.append((i_begin_segment, i_end_segment))
            # for last frame: if it is part of last segment, it has to be added to that segment.
            # If it is a new segment, this new segment of length 1 has to be added
            elif i == len(included_frames_current_round) - 1:
                if included_frames_current_round[i]:
                    if included_frames_current_round[i-1]:
                        i_end_segment = i
                        included_segments.append((i_begin_segment, i_end_segment))
                else:
                    included_segments.append((i, i))

        temp_min = np.min(temporal_current_round)
        temp_max = np.max(temporal_current_round)
        speed_min = np.min(speed_current_round)
        speed_max = np.max(speed_current_round)
        axs[1].hlines(y=[offset-0.05 for i in range(len(included_segments))], xmin=[included_segments[i][0] for i in range(len(included_segments))], xmax=[included_segments[i][1] for i in range(len(included_segments))], linewidth=3 )
        # plot the locomotion velocity
        if speed_max != speed_min:
            axs[1].plot((speed_current_round - speed_min)/(speed_max - speed_min) + offset, color="grey")
        else:  # flat line, should be simply offset
            axs[1].plot((speed_current_round - speed_min) + offset, color="grey")
        offset += 1.1
        # plot scaled calcium trace
        if temp_max != temp_min:
            axs[1].plot((temporal_current_round - temp_min)/(temp_max - temp_min) + offset, color="lightgreen")
        else:  # flat line
            axs[1].plot((temporal_current_round - temp_min) + offset, color="lightgreen")
        # plot all firing events that were recorded
        axs[1].vlines(x=firing_current_round, ymin=offset-0.05, ymax=offset+1.05, linewidth=1, color=[cmap.colors[i_color] for i_color in i_event_colors])
        offset += 1.1
    plt.tight_layout()
    plt.show()

In [ ]:
interact(plot_debug, i_cell=widgets.IntSlider(min=0, max=n_components-1, step=1, value=0))

### Calculate original and shuffled mean place coding vector lengths

In [ ]:
angles_on_belt = np.linspace(0, 2*np.pi, n_bins, endpoint=False) #initializing the circle with 150 bins corresponding to the distance on the belt
assert len(angles_on_belt) == n_bins

### Tuned vector analysis

In [ ]:
p_values_tuned = np.zeros(n_units)
shuffled_tuned_vector_lengths = np.zeros((n_units, analysis_params.n_shuffle), dtype=np.float64)
tuned_vector_lengths = np.zeros(n_units, dtype=np.float64)
for i_cell in range(n_units):
    cell_spiking = binary_spiking[i_cell]
    n_peaks=np.sum(cell_spiking)  # number of bins with firing activity TODO: make_binary(): should sum up the number of peaks in a spatial bin? Right now, it is 0 or 1
    if n_peaks == 0:
        continue
    # Calculate original mean place coding vector length
    i_rounds, i_bins = np.where(cell_spiking == 1)  # first array in tuple gives back row, second the column indices where element == 1
    event_angles = angles_on_belt[i_bins]
    event_radii = np.ones_like(event_angles)
    radius_sum, angle_sum = vector_sum(event_radii, event_angles)    
    mean_length = radius_sum/n_peaks  # normalize by number of vectors that was added up
    mean_angle = angle_sum
    tuned_vector_lengths[i_cell] = mean_length

    # Mean vector length for shuffled data
    mean_angles_shuffled = np.zeros(analysis_params.n_shuffle, dtype=np.float64)
    mean_lengths_shuffled = np.zeros(analysis_params.n_shuffle, dtype=np.float64)
    for i_shuffle in range(analysis_params.n_shuffle):
        #shuffling every row separately
        cell_spiking_shuffled = cell_spiking.copy()  # Make a copy to avoid modifying original data
        for spiking_current_round in cell_spiking_shuffled:
            np.random.shuffle(spiking_current_round)  # Shuffle the spiking events within each round independently
        # Calculate mean direction and magnitude
        i_rounds, i_bins = np.where(cell_spiking_shuffled == 1)  # first array in tuple gives back row, second the column indices where element == 1
        event_angles = angles_on_belt[i_bins]
        event_radii = np.ones_like(event_angles)
        radius_sum, angle_sum = vector_sum(event_radii, event_angles)    
        mean_lengths_shuffled[i_shuffle] = radius_sum/n_peaks  # normalize by number of vectors that was added up
        mean_angles_shuffled[i_shuffle] = angle_sum
    shuffled_tuned_vector_lengths[i_cell] = mean_lengths_shuffled
    # calculate p-value as the percentile in which the candidate cell lies
    # large p_value (>0.95) signifies place coding
    p_value = np.sum(mean_lengths_shuffled <= mean_length) / analysis_params.n_shuffle
    p_values_tuned[i_cell] = p_value

### Kolmogorov-Smirnov test
Compare the measured data to a random shuffle (baseline). Then compare n_shuffle shuffles to the same baseline.

In [ ]:
p_values_ks = np.zeros(n_units)
shuffled_data_ks = np.zeros((n_units, analysis_params.n_shuffle), dtype=np.float64)
shuffled_bl_ks = np.zeros(n_units, dtype=np.float64)
for i_cell in range(n_units):  # n_units
    cell_spiking = binary_spiking[i_cell]
    n_peaks=np.sum(cell_spiking)  # number of bins with firing activity TODO: make_binary(): should sum up the number of peaks in a spatial bin? Right now, it is 0 or 1
    if n_peaks == 0:
        continue
    # acquire baseline
    cell_spiking_bl = np.zeros(cell_spiking.shape)
    for i_round in range(len(cell_spiking_bl)):
            shift = random.randint(1, analysis_params.n_bins)
            cell_spiking_bl[i_round] = np.roll(cell_spiking[i_round], shift)  # Shuffle the spiking events within each round independently
    experiment_avg = np.mean(cell_spiking, axis=0)
    baseline_avg=np.mean(cell_spiking_bl,axis=0)
    ks_baseline,_=kstest(experiment_avg,baseline_avg)
    shuffled_bl_ks[i_cell] = ks_baseline
    # shuffle
    ks_shuffled = np.zeros(analysis_params.n_shuffle)
    for i_shuffle in range(analysis_params.n_shuffle):
        cell_spiking_shuffled = np.zeros(cell_spiking.shape)
        for i_round in range(len(cell_spiking_shuffled)):
            shift = random.randint(1, analysis_params.n_bins)  # TODO: shift lower range should be 1 or 0?
            cell_spiking_shuffled[i_round] = np.roll(cell_spiking[i_round], shift)  # Shuffle the spiking events within each round independently
        shuffled_avg = np.mean(cell_spiking_shuffled, axis=0)
        ks_shuffle,p_value_=kstest(baseline_avg, shuffled_avg)
        ks_shuffled[i_shuffle] = ks_shuffle
    shuffled_data_ks[i_cell] = ks_shuffled
    # calculate p value
    p_value_ks = np.sum(ks_shuffled > ks_baseline) / len(ks_shuffled)
    p_values_ks[i_cell] = p_value_ks

# Plotting

In [ ]:
# TODO: make an ultimate debug plot:
# plot the trace per each round, and underline the frames that are included for analysis (when I do cuts, also cut for the frame indices in the end).
# Plot locomotion as well, to see that only frames with locomotion are included. Also plot the binary spikes. Then plot the spatial bins too? 
# As vlines or different colored spikes for each spatial bin?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Load data into dataframes

### Firing rate per bin
Columns:  cell no., bin index, avg firing rate over rounds, p_tuning, p_ks

Firing rate per cell per round per bin

In [ ]:
n_entries = frm_z.flatten().shape[0]
cell_ids = np.zeros(frm_z.shape, dtype=np.int16)
rounds = np.zeros(frm_z.shape, dtype=np.int16)
bin_idxs = np.zeros(frm_z.shape, dtype=np.int16)
# TODO loop over cells, rounds, and bins, add cell_id, round, bin_idx, firing_rate
for i_cell in range(n_units):
    cell_ids[i_cell] = np.full((frm_z.shape[1], frm_z.shape[2]), i_cell, dtype=np.int16)
    for i_round in range(frm_z.shape[1]):
        rounds[i_cell][i_round] = np.full(frm_z.shape[2], i_round, dtype=np.int16)
        bin_idxs[i_cell][i_round] = np.linspace(0, frm_z_avg.shape[1], num=frm_z_avg.shape[1], endpoint=False, dtype=np.int16)
dict_fr={"cell_id":cell_ids.flatten(), "round":rounds.flatten(), "bin_idx":bin_idxs.flatten(), "firing_rate":binary_spiking.flatten()}
df_firing_rate = pd.DataFrame(dict_fr)

Averaged firing rate over rounds

In [ ]:
n_entries = frm_z_avg.flatten().shape[0]
cell_ids = np.zeros(frm_z_avg.shape, dtype=np.int16)
bin_idxs = np.zeros(frm_z_avg.shape, dtype=np.int16)
# TODO loop over cells, rounds, and bins, add cell_id, round, bin_idx, firing_rate
for i_cell in range(n_units):
    cell_ids[i_cell] = np.full(frm_z_avg.shape[1], i_cell)
    bin_idxs[i_cell] = np.linspace(0, frm_z_avg.shape[1], num=frm_z_avg.shape[1], endpoint=False, dtype=np.int16)
dict_fr={"cell_id":cell_ids.flatten(), "bin_idx":bin_idxs.flatten(), "firing_rate":frm_z_avg.flatten()}
df_avg_firing_rate = pd.DataFrame(dict_fr)

### P values per cell for both methods and bin index for first maximum of firing rate

In [ ]:
dict_p_vals = {"cell_id": np.array([i for i in range(n_units)]), "i_fr_max": np.argmax(frm_z_avg, axis=1), "p_tuning": p_values_tuned, "p_ks": p_values_ks}
df_p_values = pd.DataFrame(data=dict_p_vals)
del dict_p_vals


### Join the dataframes

In [ ]:
df_avg_firing_rate = df_avg_firing_rate.join(df_p_values, how="left", on="cell_id", rsuffix="_other")

## Plot firing rate over belt for place cells vs non-place cells

### Tuned vector method

In [ ]:
pivot_firing_rate_accepted = df_avg_firing_rate[df_avg_firing_rate["p_tuning"] > 0.95].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)
pivot_firing_rate_rejected = df_avg_firing_rate[df_avg_firing_rate["p_tuning"] <= 0.95].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)

f, axs = plt.subplots(1, 2, figsize=(18, 6))
sns.heatmap(pivot_firing_rate_accepted, ax=axs[0])
sns.heatmap(pivot_firing_rate_rejected, ax=axs[1])

axs[0].set_title("accepted")
axs[1].set_title("rejected")
plt.show()

### KS method

In [ ]:
pivot_firing_rate_accepted = df_avg_firing_rate[df_avg_firing_rate["p_ks"] > 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)
pivot_firing_rate_rejected = df_avg_firing_rate[df_avg_firing_rate["p_ks"] <= 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)

f, axs = plt.subplots(1, 2, figsize=(18, 6))
sns.heatmap(pivot_firing_rate_accepted, ax=axs[0])
sns.heatmap(pivot_firing_rate_rejected, ax=axs[1])
axs[0].set_title("accepted")
axs[1].set_title("rejected")

plt.show()

## Cell-specific figures

In [ ]:
i_cell = 10

### Activity per round

In [ ]:
#pivot_firing_rate_accepted = df_avg_firing_rate[df_avg_firing_rate["p_ks"] > 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)
pivot_table_single = df_firing_rate[df_firing_rate["cell_id"] == i_cell].pivot_table(columns="bin_idx", index="round", values="firing_rate")
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(pivot_table_single, ax=ax)
ax.set_title(f"Cell #{i_cell}")

plt.show()

### Shuffle histograms

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12, 4))
axs[0].hist(shuffled_tuned_vector_lengths[i_cell])
axs[1].hist(shuffled_data_ks[i_cell])

axs[0].vlines(x=tuned_vector_lengths[i_cell], ymin=0, ymax=100, color="red")
axs[1].vlines(x=shuffled_bl_ks[i_cell], ymin=0, ymax=100, color="red")

plt.show()

### Circular plotting of activity

In [ ]:
i_cell = 359
binary_spiking_cell = binary_spiking[i_cell]
n_bins = len(binary_spiking_cell[0])
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.set_xlabel('Angle (degrees)')
ax.set_ylabel('Radius')
for i_round in range(len(binary_spiking_cell)):
    i_firing_bins = binary_spiking_cell[i_round].nonzero()[0]
    if len(i_firing_bins) > 0:
        vector_length = i_round + 1  # avoid 0 length
        vector_lengths = [vector_length]*len(i_firing_bins)
        firing_angles_deg = i_firing_bins*360./n_bins
        ax.scatter(firing_angles_deg, vector_lengths)
plt.show()

# Interactive plotting

In [ ]:
def plot_data(i_cell):
    pivot_table_single = df_firing_rate[df_firing_rate["cell_id"] == i_cell].pivot_table(columns="bin_idx", index="round", values="firing_rate")
    fig = plt.figure(figsize=(12, 12))
    binary_spiking_cell = binary_spiking[i_cell]
    n_bins = len(binary_spiking_cell[0])
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(222, projection='polar')
    ax3 = plt.subplot(223)
    ax4  =plt.subplot(224)
    sns.heatmap(pivot_table_single, ax=ax1)
    ax2.set_xlabel('Angle (degrees)')
    ax2.set_ylabel('Round')
    for i_round in range(len(binary_spiking_cell)):
        i_firing_bins = binary_spiking_cell[i_round].nonzero()[0]
        if len(i_firing_bins) > 0:
            vector_length = i_round + 1  # avoid 0 length
            vector_lengths = [vector_length]*len(i_firing_bins)
            firing_angles_deg = i_firing_bins*360./n_bins
            ax2.scatter(firing_angles_deg, vector_lengths)
    ax2.set_ylim((0, len(binary_spiking_cell)+0.2))
    ax1.set_title(f"Cell #{i_cell}")
    ax3.hist(shuffled_tuned_vector_lengths[i_cell], bins=20)
    ax4.hist(shuffled_data_ks[i_cell], bins=20)
    ax3.vlines(x=tuned_vector_lengths[i_cell], ymin=0, ymax=100, color="red")
    ax4.vlines(x=shuffled_bl_ks[i_cell], ymin=0, ymax=100, color="red")
    plt.show()

In [ ]:
interact(plot_data, i_cell=widgets.IntSlider(min=df_firing_rate['cell_id'].min(), max=df_firing_rate['cell_id'].max(), step=1, value=df_firing_rate['cell_id'].min()))

# Export results

In [ ]:
fpath_output = os.path.join(output_folder, os.path.splitext(os.path.split(fpath_expinfo)[-1])[0].replace("expinfo", "placecoding")+".h5")
print(f"Saving to {fpath_output}")

In [ ]:
if save_results:
    with h5py.File(fpath_output, "w") as hf:
        dict_analysis_params = analysis_params.to_dict()
        for key in dict_analysis_params.keys():
            hf.attrs[key] = dict_analysis_params[key]
        hf.create_dataset("frm_raw", data=frm_raw)
        hf.create_dataset("frm_z", data=frm_z)
        hf.create_dataset("binary_spiking", data=binary_spiking)
        hf.create_dataset("tuned_vector_lengths", data=tuned_vector_lengths)
        hf.create_dataset("shuffled_tuned_vector_lengths", data=shuffled_tuned_vector_lengths)
        hf.create_dataset("p_values_tuned", data=p_values_tuned)
        hf.create_dataset("shuffled_data_ks", data=shuffled_data_ks)
        hf.create_dataset("shuffled_bl_ks", data=shuffled_bl_ks)
        hf.create_dataset("p_values_ks", data=p_values_ks)
        # TODO: add spatial components (to make it a standalone file) as sparse matrix
    print(f"Saved to {fpath_output}")
